# Conversão de Vento (2 m → 10 m)

> Notebook organizado para reprodutibilidade. Edite apenas a célula **CONFIGURAÇÕES**.

In [ ]:
from pathlib import Path
import os

# CONFIGURAÇÕES (edite se necessário)
# A pasta raiz do projeto (por padrão, a pasta acima de /notebooks)
ROOT = Path(os.getenv('CLIMBRA_PROJECT_ROOT', Path.cwd().parent)).resolve()
DATA_DIR = ROOT / 'data'
RAW_DIR  = DATA_DIR / '00_raw'
INT_DIR  = DATA_DIR / '01_intermediate'
FINAL_DIR= DATA_DIR / '02_final'
OUT_DIR  = ROOT / 'outputs'
FIG_DIR  = OUT_DIR / 'figures'
TAB_DIR  = OUT_DIR / 'tables'

for d in [RAW_DIR, INT_DIR, FINAL_DIR, FIG_DIR, TAB_DIR]:
    d.mkdir(parents=True, exist_ok=True)


In [ ]:
# -*- coding: utf-8 -*-
"""
Conversão de vento de 2 m -> 10 m usando a formulação FAO-56.

Percorre recursivamente todas as pastas em:
    E:\CLIMBRA_SSP245\wind_input

Mantém a mesma estrutura em:
    E:\CLIMBRA_SSP245\wind_output

Formato esperado do CSV (exemplo):
    dia;mes;ano;vento   (vento = u2 em m/s)

Saída contém:
    u2   (vento original)
    u10  (vento convertido)
"""

import re
from pathlib import Path
import pandas as pd
import numpy as np

# =========================
# CONFIGURAÇÕES
# =========================

INPUT_DIR = RAW_DIR / 'sfc_input'
OUTPUT_DIR = OUT_DIR / 'sfc_output'

# Nome da coluna com o vento a 2 metros
WIND_COLUMN = "valor"

# Sufixo do arquivo convertido
OUTPUT_SUFFIX = "_u10"

# =========================
# FUNÇÃO DE CONVERSÃO
# =========================

def wind_2_to_10(u2):
    """
    Converte vento de 2 m para 10 m (m/s)
    Fórmula FAO-56:
        u10 = u2 * ln(67.8*10 - 5.42) / 4.87
        ln(672.58)/4.87 = 1.336
    """
    return u2 * 1.336

# =========================
# PROCESSAMENTO DE UM ARQUIVO
# =========================

def process_file(path_in: Path, base_in: Path, base_out: Path):
    print(f"Processando: {path_in}")

    df = pd.read_csv(path_in, sep=";")

    if WIND_COLUMN not in df.columns:
        raise KeyError(f"Coluna '{WIND_COLUMN}' não encontrada em {path_in}")

    # vento original (2 m)
    u2 = df[WIND_COLUMN].astype(float)

    # cálculo do vento a 10 m
    u10 = wind_2_to_10(u2)

    # monta saída
    df_out = df.copy()
    df_out["u2"] = u2
    df_out["u10"] = u10

    # mantém estrutura de pastas
    rel_parent = path_in.relative_to(base_in).parent
    out_dir = base_out / rel_parent
    out_dir.mkdir(parents=True, exist_ok=True)

    # salva arquivo
    out_name = path_in.stem + OUTPUT_SUFFIX + path_in.suffix
    path_out = out_dir / out_name
    df_out.to_csv(path_out, sep=";", index=False)

    print(f"  -> Salvo em: {path_out}")

# =========================
# SCRIPT PRINCIPAL
# =========================

def main():
    base_in = Path(INPUT_DIR)
    base_out = Path(OUTPUT_DIR)
    base_out.mkdir(parents=True, exist_ok=True)

    files = sorted(base_in.rglob("*.csv"))

    if not files:
        print(f"Nenhum arquivo encontrado em {base_in}")
        return

    print(f"Encontrados {len(files)} arquivo(s).\n")

    for i, file in enumerate(files, start=1):
        try:
            print(f"[{i}/{len(files)}]")
            process_file(file, base_in, base_out)
        except Exception as e:
            print(f"ERRO ao processar {file}: {e}\n")

if __name__ == "__main__":
    main()